In [58]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from gurobipy import GRB, Model

In [59]:
static = pd.read_csv('static_with_t.csv')
detected = pd.read_csv('detection_graph.csv')

In [60]:
level_order = [
    "root_cross_right",
    "root_cross_left",
    "root_cross_right2",
    "conn_k",
    "cross_e",
    "cross_bag_br",
    "cross_ij1",
    "cross_e_c",
    "cross_f",
    "bag_b_r",
    "bag_a_r",
    "conn_b",
    "cross_ij2",
    "conn_e",
    "conn_c",
    "cross_k",
    "cross_f30",
    "conn_i",
    "conn_j",
    "box",
    "conn_a",
    "conn_f_30",
    "cross_f37",
    "conn_f_60",
    "bag_b_l",
    "cross_gh",
    "conn_f_37",
    "conn_d",
    "conn_g",
    "conn_h",
    "bag_a_l",
]

In [61]:
quadratic_model = Model('quadratic')

In [62]:
variables = {}
for name in level_order:
    variables[f"{name}_x"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_x"
    )
    variables[f"{name}_y"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_y"
    )
    variables[f"{name}_z"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_z"
    )
    
    
        

In [63]:
obj_fn = 0
for name in level_order:
    if name == "root":
        continue
    node = detected[detected["name"] == name]
    if node.shape[0] != 0:
        detected_x = node["x"].values[0]
        detected_y = node["y"].values[0]
        detected_z = node["z"].values[0]
        parent = static[static["name"] == name]["parent"].values[0]
        term = (
            (variables[f"{name}_x"] + variables[f"{parent}_x"] - detected_x) ** 2 #do poprawy
            + (variables[f"{name}_y"] + variables[f"{parent}_y"] - detected_y) ** 2
            + (variables[f"{name}_z"] + variables[f"{parent}_z"] - detected_z) ** 2
        )
        obj_fn += term

In [64]:
quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)


In [65]:
quadratic_model.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 93 columns and 0 nonzeros
Model fingerprint: 0x1612432f
Model has 132 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [6e+00, 9e+02]
  QObjective range [2e+00, 6e+00]
  Bounds range     [2e+02, 2e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 15 columns
Presolve time: 0.01s
Presolved: 0 rows, 78 columns, 0 nonzeros
Presolved model has 132 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 54
 AA' NZ     : 4.500e+01
 Factor NZ  : 9.900e+01
 Factor Ops : 2.250e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual  

In [66]:
#print all variable values
for v in quadratic_model.getVars():
    print(v.varName, v.x)

root_cross_right_x 10.0
root_cross_right_y -25.0
root_cross_right_z -4.5
root_cross_left_x -250.0
root_cross_left_y -250.0
root_cross_left_z -250.0
root_cross_right2_x -250.0
root_cross_right2_y -250.0
root_cross_right2_z -250.0
conn_k_x 0.191379526690298
conn_k_y -89.04104632774133
conn_k_z -13.197018918015203
cross_e_x -250.0
cross_e_y -250.0
cross_e_z -250.0
cross_bag_br_x 12.50520148254816
cross_bag_br_y -85.26539211135861
cross_bag_br_z -1.2501307560659143
cross_ij1_x -250.0
cross_ij1_y -250.0
cross_ij1_z -250.0
cross_e_c_x -2.666702122154078
cross_e_c_y -58.00061498500102
cross_e_c_z -7.999975702345978
cross_f_x -250.0
cross_f_y -250.0
cross_f_z -250.0
bag_b_r_x 3.4947985174518124
bag_b_r_y -19.73460788864142
bag_b_r_z -3.7498692439340857
bag_a_r_x 0.49479851745178394
bag_a_r_y -64.73460788864136
bag_a_r_z 1.2501307560659427
conn_b_x 8.49479851745184
conn_b_y -0.7346078886414489
conn_b_z 1.2501307560659427
cross_ij2_x 12.671296899198353
cross_ij2_y -49.35552573134933
cross_ij2_z 

In [67]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from transformation_utils import (
    get_rotation,
    rotate_graph,
    get_rotation_nn,
    optim_rotation,
)

G = nx.Graph()

for name in level_order:
    # if x and y and z are  different than 250 or -250
    if variables[f"{name}_x"].x != 250 and variables[f"{name}_x"].x != -250:
        # print(name, variables[f"{name}_x"].x, variables[f"{name}_y"].x, variables[f"{name}_z"].x)
        G.add_node(
            name,
            pos=(
                variables[f"{name}_x"].x,
                variables[f"{name}_z"].x,
                variables[f"{name}_y"].x,
            ),
        )

plot_graph(static_graph=G, detected_graph=None, name="qcqp2", save=True)

Graph saved to qcqp2.html
